In [1]:
import os
from pyspark.sql import SparkSession

os.environ['JAVA_HOME'] = '/usr/lib/jvm/zulu-8'
os.environ['PYSPARK_PYTHON'] = '/home/debian1/BigData_UPAO/bigdata_env/bin/python3'
os.environ['PYSPARK_DRIVER_PYTHON'] = '/home/debian1/BigData_UPAO/bigdata_env/bin/python3'
os.environ['HADOOP_CONF_DIR'] = '/opt/hadoop-3.3.6/etc/hadoop'
os.environ['YARN_CONF_DIR'] = '/opt/hadoop-3.3.6/etc/hadoop'

#RUTA DEL DRIVER
jar_path = "/home/debian1/BetGol/postgresql-42.7.3.jar"

# INICIAR SPARK CON EL DRIVER
spark = (
    SparkSession.builder
    .appName("BetGol-Visualizacion")
    .master("yarn")
    .config("spark.jars", jar_path)
    .config("spark.driver.extraClassPath", jar_path)
    .config("spark.driver.memory", "3g")
    .config("spark.executor.memory", "3g")
    .config("spark.executor.cores", "2")
    .config("spark.sql.shuffle.partitions", "150")
    .config("spark.sql.parquet.compression.codec", "snappy")
    .getOrCreate()
)

print("Spark iniciado correctamente con soporte para PostgreSQL")
print(f"Driver cargado desde: {jar_path}")
print("Versión:", spark.version)


25/11/24 04:55:35 WARN Utils: Your hostname, vbox resolves to a loopback address: 127.0.1.1; using 192.168.18.70 instead (on interface enp0s3)
25/11/24 04:55:35 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
25/11/24 04:55:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/24 04:55:40 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


Spark iniciado correctamente con soporte para PostgreSQL
Driver cargado desde: /home/debian1/BetGol/postgresql-42.7.3.jar
Versión: 3.5.1


In [2]:
# Cargar los modelos desde HDFS (FORMATO CSV)

print("--- CARGANDO RESULTADOS DESDE HDFS ---")

# Cargar Over/Under
df_over25 = spark.read.csv(
    "hdfs:///user/johan/salidas_csv/predicciones_over25/", 
    header=True, 
    inferSchema=True  # Importante para que detecte números y no todo como texto
)

# Cargar 1x2
df_1x2 = spark.read.csv(
    "hdfs:///user/johan/salidas_csv/predicciones_1x2/", 
    header=True, 
    inferSchema=True
)

print(f"Cargados: {df_over25.count()} filas de Goles y {df_1x2.count()} filas de Ganador.")


--- CARGANDO RESULTADOS DESDE HDFS ---


Cargados: 3184 filas de Goles y 3183 filas de Ganador.


## Conexion a la bd

In [4]:
# Configuración JDBC para Postgres
db_properties = {
    "user": "admin",
    "password": "1234",
    "driver": "org.postgresql.Driver"
}
url_db = "jdbc:postgresql://localhost:5432/futbol_db"

# Escribir en Postgres
print("--- ESCRIBIENDO EN POSTGRESQL ---")
df_over25.write.jdbc(url=url_db, table="predicciones_over25", mode="overwrite", properties=db_properties)
df_1x2.write.jdbc(url=url_db, table="predicciones_1x2", mode="overwrite", properties=db_properties)

print("¡Datos exportados a la BD!")


--- ESCRIBIENDO EN POSTGRESQL ---


¡Datos exportados a la BD!
